In [2]:
import pandas as pd
import sys
import matplotlib.pyplot as plt
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize,RegexpTokenizer
import string
import nltk
from nltk.corpus import stopwords
import en_core_web_sm
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [4]:
rev=pd.read_csv('yelp/yelp_review_exp.csv',encoding='UTF-8')

/home/vishwa/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
filt_condition='user_id'
filt_variable='CxDOIDnH8gp9KXzpBHJYXw'

In [6]:
rev1=rev[rev.cat=='Restaurants Thai']

In [7]:
rev1['tokenized_text'] = rev1['text'].apply(lambda x:x.lower()).apply(word_tokenize) 
punctuation = list(string.punctuation)
stopWords = set(stopwords.words('english'))
docs=list(rev1['tokenized_text'])


/home/vishwa/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
filtered_docs=[]
for i in docs:
    filt = []
    for w in i:
        if w.lower()not in stopWords and w not in punctuation:
            filt.append(w)
    filtered_docs.append(filt)

nlp = en_core_web_sm.load()

final_txt=filtered_docs

In [9]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [10]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(final_txt)]

In [11]:
model = Doc2Vec(dm = 1, min_count=2, window=10, size=150, sample=1e-4, negative=5)
model.build_vocab(documents)
model

/home/vishwa/anaconda2/lib/python2.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [12]:
for epoch in range(10):
    model.train(documents,epochs=model.iter,total_examples=model.corpus_count)
    print("Epoch #{} is complete.".format(epoch+1))

/home/vishwa/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


Epoch #1 is complete.
Epoch #2 is complete.
Epoch #3 is complete.
Epoch #4 is complete.
Epoch #5 is complete.
Epoch #6 is complete.
Epoch #7 is complete.
Epoch #8 is complete.
Epoch #9 is complete.
Epoch #10 is complete.


In [13]:
ll=[]
for i in range(len(rev1)):
    ll.append(list(model.docvecs[i]))

In [14]:
cols=[]
for i in range(150):
    cols.append('dim_'+str(i))

In [15]:
df=pd.DataFrame.from_records(ll,columns=cols,index=rev1.index)

In [16]:
#dd=pd.concat([df,rev[['useful','cool','funny']]],axis=1)
dd=df.join(rev1[['useful','cool','funny']])

In [17]:
dd['target']=np.where(dd['useful']>0, 1, 0)

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, accuracy_score,recall_score,roc_auc_score,f1_score,classification_report
from sklearn.model_selection  import GridSearchCV
from sklearn.model_selection  import KFold, StratifiedKFold
from sklearn.model_selection  import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import make_scorer, accuracy_score,recall_score,roc_auc_score

In [19]:
X=dd[cols]
X=X
Y=dd.target
Y=Y

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=168)

In [21]:
from xgboost import XGBClassifier

In [22]:
xgb_clf = XGBClassifier(learning_rate =0.1,
 n_estimators=100,
 max_depth=3,
 min_child_weight=5,
 gamma=0.2,
 subsample=0.6,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 seed=27)
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.2, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=5, missing=None,
       n_estimators=100, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.6)

In [23]:
predictions = xgb_clf.predict(X_test)
print "Area under curve for Gradient Boosting predection scores-",roc_auc_score(y_test, predictions)

Area under curve for Gradient Boosting predection scores- 0.596960138539754


In [24]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='rbf',C=10,gamma=.1)
svclassifier.fit(X_train, y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [25]:
predictions = svclassifier.predict(X_test)
print "Area under curve for SVM predection scores-",roc_auc_score(y_test, predictions)

Area under curve for SVM predection scores- 0.595729561853437
